In [1]:
import warnings
warnings.filterwarnings('ignore')

MY_API_TOKEN = ""
TEST_GRACEID = "MS181101ab"

### The Treasure Map API: Candidates


### Candidate model
The `gwtm_api.Candidate` model covers the absolute basic parameters of a photometric astronomical observation. But most importantly it requires an instrument footprint to already be established in the Treasure Map. This can be accomplished by having a valid account, and going to the [Submit Instrument](https://treasuremap.space/submit_instrument) page, and following the instructions there. 

The `gwtm_api.Candidate` fields are as follows:

- `ra: float` - the Right Ascenscion (RA) of the pointing.
- `dec: float` - the Declination (DEC) of the pointing.
  - The `ra` and `dec` values can also be input as `position: str` that has the strict format of `"POINT (ra dec)"`.
- `graceid: str` - the GW event graceid
- `candidate_name: str` - the name of the candidate (not required)
- `tns_name: str` - the name associataed if submitted to the Transient Name server (not required)
- `tns_url: str` - the url associated to the TNS (not required)
- `associated_galaxy: str` - the name of associated galaxy (not required)
- `associated_galaxy_redshift: float` - the redshift of the associated galaxy (not required)
- `associated_galaxy_distance: float` - the distance of the associated galaxy (not required)
- `discovery_date: str | datetime` - the UTC time associated with the candidate discovery. The string format of the time `%Y-%m-%dT%H:%M:%S.%f` format. e.g. `"2019-05-01T12:00:00.00"`. Otherwise `datetime.datetime` will be converted to that format. 
- `discovery_magnitude: float` - the apparent magnitude of the candidate at discovery
- `magnitude_unit: str | gwtm_api.enums.depth_unit` - the discovery magnitude unit. Can be: `ab_mag`, `vega_ma"`, `flux_erg`, or `flux_jy`.
- **Bandpass Information** can be passed in by stating the band, or the range of wavelengths, energies, or frequencies the pointing covers. If your pointing is associated with a bandpass, we will calculate the wavelength range from a generic set of filters from [SVO](http://svo2.cab.inta-csic.es/theory/fps/index.php?asttype=astro)
  - *Method 1*: Bandpass list
    - `magnitude_bandpass: str | gwtm_api.enums.bandpass` - the name of the bandpass. Can be one of: `U`, `B`, `V`, `R`, `I`, `J`, `H`, `K`, `u`, `g`, `r`, `i`, `z`, `UVW1`, `UVW2`, `XRT`, `clear`, `open`, `UHF`, `VHF`, `L`, `S`, `C`, `X`, `other`, `TESS`, `BAT`, `HESS`, `WISEL`, and `q`
  - *Method 2*: Central Wavelength and Bandwidth
    - `magnitude_central_wave: float` - the central wavelength in Angstroms
    - `magnitude_bandwidth: float` - the bandwidth in Angstroms
  - *Method 3*: Spectrum Regimes: There are 3 ways to submit a spectrum regime for your associated pointing. By stating the wavelength, energy, or frequency regimes along with their associated units
    - `wavelength_regime: list[float]` - the begin and endpoints of the wavelength bandwidth e.g. `[low, high]`
    - `wavelength_unit: str | gwtm_api.enums.wavelength_units` - valid inputs are `angstrom`, `nanometer` or `micron`
    - or `energy_regime: list[float]` - the begin and endpoints of the energy bandwidth e.g. `[low, high]`
    - with `energy_unit: str | gwtm_api.enums.energy_units` - valid inputs are: `eV`, `keV`, `MeV`, `GeV`, and `TeV`
    - or `frequency_regime: list[float]` - the begin and endpoints of the frequency bandwidth e.g, `[low, high]`
    - with `frequench_unit: str | gwtm_api.enums.frequency_units` - valid inputs are: `Hz`, `kHz`, `MHz`, `GHz` and `THz`

### Using the gwtm_api.Candidate api_wrapper.
##### Instantiating a single Candidate and posting

In [2]:
import datetime
from gwtm_api import Candidate
from gwtm_api import enums as gwenums

my_candidate = Candidate(
    graceid=TEST_GRACEID,
    candidate_name="AT2017gfo",
    tns_name="AT2017gfo",
    tns_url="https://www.wis-tns.org/object/2017gfo",
    ra=197.4503,
    dec=-23.3815,
    discovery_date=datetime.datetime(2017, 8, 17, 23, 31, 12),
    discovery_magnitude=17.3,
    magnitude_unit=gwenums.depth_unit.ab_mag,
    magnitude_bandpass=gwenums.bandpass.i,
    associated_galaxy="NGC 4993",
    associated_galaxy_redshift=0.009727
)
my_candidate.dump()

position: POINT (197.4503 -23.3815)
graceid: MS181101ab
candidate_name: AT2017gfo
tns_name: AT2017gfo
tns_url: https://www.wis-tns.org/object/2017gfo
ra: 197.4503
dec: -23.3815
discovery_date: 2017-08-17 23:31:12
discovery_magnitude: 17.3
magnitude_bandpass: 12
magnitude_unit: AB mag
associated_galaxy: NGC 4993
associated_galaxy_redshift: 0.009727



In [4]:
#post a single candidate
my_candidate.post(api_token=MY_API_TOKEN, verbose=True)

Exception: It seems like you are trying to rePOST this record

In [5]:
my_candidate.dump()

position: POINT (-162.5497 -23.3815)
graceid: MS181101ab
candidate_name: AT2017gfo
tns_name: AT2017gfo
tns_url: https://www.wis-tns.org/object/2017gfo
ra: -162.5497
dec: -23.3815
discovery_date: 2017-08-17 23:31:12
discovery_magnitude: 17.3
magnitude_bandpass: i
magnitude_unit: ab_mag
associated_galaxy: NGC 4993
associated_galaxy_redshift: 0.009727
id: 40
datecreated: 2024-05-10 22:35:55.984768
submitterid: 2
magnitude_central_wave: 7836.21
magnitude_bandwidth: 1468.29
associated_galaxy_distance: None



##### Batch Candidate Submission
Here we will give examples of instantiating multiple Candidate objects and posting the list in a single request

```python
batch = [
    Candidate(),
    Candidate()
]
Candidate.batch_post(candidates=batch, api_token="...", graceid="...")
```

In [8]:
batch = [
    Candidate(
        position="POINT (24 48)",
        candidate_name="candidate_name1",
        discovery_date="2024-05-04T12:12:12.12Z",
        discovery_magnitude=17.5,
        magnitude_unit="ab_mag",
        magnitude_bandpass="r",
    ),
    Candidate(
        ra=24,
        dec=48,
        candidate_name="candidate_name2",
        discovery_date="2024-05-05T12:12:12.12Z",
        discovery_magnitude=12e-10,
        magnitude_unit="flux_erg",
        energy_regime=[0.1, 10],
        energy_unit=gwenums.energy_units.keV
    )
]

batch = Candidate.batch_post(candidates=batch, api_token=MY_API_TOKEN, graceid=TEST_GRACEID, verbose=True)

{'candidate_ids': [43, 44], 'ERRORS': [], 'WARNINGS': []}


In [10]:
for b in batch:
    b.dump()

position: POINT (24 48)
id: 43
datecreated: 2024-05-10 22:49:45.187281
submitterid: 2
graceid: MS181101ab
candidate_name: candidate_name1
tns_name: 
tns_url: 
discovery_date: 2024-05-04 12:12:12.120000
discovery_magnitude: 17.5
magnitude_central_wave: 6415.4
magnitude_bandwidth: 1487.58
magnitude_unit: ab_mag
magnitude_bandpass: r
associated_galaxy: 
associated_galaxy_redshift: None
associated_galaxy_distance: None
ra: 24.0
dec: 48.0

position: POINT (24 48)
id: 44
datecreated: 2024-05-10 22:49:45.187699
submitterid: 2
graceid: MS181101ab
candidate_name: candidate_name2
tns_name: 
tns_url: 
discovery_date: 2024-05-05 12:12:12.120000
discovery_magnitude: 1.2e-11
magnitude_central_wave: 1.2398
magnitude_bandwidth: 0.0
magnitude_unit: flux_erg
magnitude_bandpass: BAT
associated_galaxy: 
associated_galaxy_redshift: None
associated_galaxy_distance: None
ra: 24.0
dec: 48.0



#### Querying for submitted candidates
Here we will give examples for utilizing the `gwtm_api.Candidate.get()` method to search for existing candidates.

In [13]:
queried_candidates = Candidate.get(api_token=MY_API_TOKEN, graceid=TEST_GRACEID)
print(len(queried_candidates))

queried_candidates[0].dump()

6
position: POINT (-162.5497 -23.3815)
id: 39
datecreated: 2024-05-10 22:33:00.866643
submitterid: 2
graceid: MS181101ab
candidate_name: AT2017gfo
tns_name: AT2017gfo
tns_url: https://www.wis-tns.org/object/2017gfo
discovery_date: 2017-08-17 23:31:12
discovery_magnitude: 17.3
magnitude_central_wave: 7836.21
magnitude_bandwidth: 1468.29
magnitude_unit: ab_mag
magnitude_bandpass: i
associated_galaxy: NGC 4993
associated_galaxy_redshift: 0.009727
associated_galaxy_distance: None
ra: -162.5497
dec: -23.3815



##### Further filtering options
- `id: int` - candidate id
- `ids: list[int]` - list of candidate ids
- `submitted_date_after: str | datetime.datetime` -  candidates created after or on a specified datetime
- `submitted_date_before: str | datetime.datetime` - candidates created before or on a specified datetime
- `discovery_date_after: str | datetime.datetime` - candidates discovered after or on a specified datetime
- `discovery_date_after: str | datetime.datetime` - candidates discovered after or on a specified datetime
- `discovery_magnitude_gt: float` - candidates with magnitude greater than (or equal) specified value
- `discovery_magnitude_lt: float` - candidates with magnitude lesser than (or equal) specified value
- `associated_galaxy_redshift_gt: float` - candidates with associated_galaxy_redshift greater than (or equal) specified value
- `associated_galaxy_redshift_lt: float` - candidates with associated_galaxy_redshift lesser than (or equal) specified value
- `associated_galaxy_distance_gt: float` - candidates with associated_galaxy_distance greater than (or equal) specified value
- `associated_galaxy_distance_lt: float` - candidates with associated_galaxy_distance lesser than (or equal) specified value

#### Altering your submitted candidates
Here we will give examples for utilizing the `gwtm_api.Candidate.put()` method to search for existing candidates.

#### Deleting you submitted candidates
Here we will give examples for utilizing the `gwtm_api.Candidate.delete()` and `gwtm_api.Candidate.batch_delete()` methods to search for existing candidates.

In [15]:
class mytest:
    pass

cali = mytest()
cali.whats_UP = "boogers"
print(f"whats up {cali.__dict__}")

whats up {'whats_UP': 'boogers'}
